In [ ]:
#%pip install pandas-datareader
%pip install yfinance
#!pip install yfinance --upgrade --no-cache-dir

In [19]:
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf

In [ ]:
pd.set_option('display.float_format', lambda x: '%.8f' % x) 
# change the visualization of a float to eight decimal digits.

In [29]:
binance = pd.read_csv('data/final/saldo_binance.csv', index_col=0)
trezor = pd.read_csv('data/final/saldo_trezor.csv', index_col=0)
binance_complete = pd.read_csv('data/final/binance_dates.csv', index_col=0)

In [34]:
tickers = list(set(binance.index).union(trezor.index).union(binance_complete.index))
tickers = ['ETH', 'MATIC', 'BTC']

In [22]:
end_date = datetime.now().strftime("%Y-%m-%d")

In [51]:
# Set the start and end dates for the historical data
start_date = (datetime.now()- timedelta(days=1)).strftime("%Y-%m-%d")
end_date = datetime.now().strftime("%Y-%m-%d")

# Fetch the historical data for each ticker using Yahoo Finance API
hist_data = {}
for ticker in tickers:
    data = yf.download(ticker + '-USD', start=start_date, end=end_date)
    hist_data[ticker] = data['Close']

# Combine the historical data for all tickers into a single dataframe
df = pd.concat(hist_data, axis=1, keys=tickers)

# Print the head of the combined dataframe
print(df.head())


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                    ETH     MATIC           BTC
Date                                           
2023-03-20  1735.321289  1.104097  27767.236328


In [52]:
df

,ETH,MATIC,BTC
Date,,,
2023-03-20,1735.321289,1.104097,27767.236328


In [55]:
data = pd.DataFrame(index=tickers)
data['Amount'] = 0


In [59]:
for token in tickers:
    print(token)
    try:
        data.loc[token,'Amount'] += trezor.loc[token,'Amount'] 
    finally:
        try:
            data.loc[token,'Amount'] += binance.loc[token,'Balance']

SyntaxError: incomplete input (621386529.py, line 7)

In [58]:
binance

,UTC_Time,Balance,BRL_spent,Mean_price
Coin,,,,
BTC,2022-01-04 19:50:53,0.002010,250.000000,124380.584688
BUSD,2022-02-19 02:28:25,0.006698,0.000000,0.000000
LUNA,2022-03-07 17:16:16,0.039680,0.000000,0.000000
MATIC,2022-08-24 14:43:07,116.414976,650.341787,5.586410
ETHW,2022-09-20 01:18:13,0.339235,0.000000,0.000000


In [50]:
data

,Amount
ETH,1.058043
MATIC,0.000000
BTC,0.000000
